In [0]:
#Bronze ingestion
#need to update section to read from volume
df_raw = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(full_path)
)

display(df_raw)

#Create Unity Catalog Table
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.bronze")

(
    df_raw.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.bronze.tennis_points_raw")
)